In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("your_groq_api_key")
groq_api_key



In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key="your_groq_api_key")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D1AA892980>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D1AA893DF0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Sarthak and I am a AI Software Engineer")])

AIMessage(content="Hi Sarthak, it's nice to meet you! \n\nThat's awesome, being an AI Software Engineer is such a fascinating field. What kind of projects are you working on these days? What are you most excited about in the world of AI?  \n\nI'm always eager to learn more about what people are doing in this space. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 24, 'total_tokens': 103, 'completion_time': 0.143636364, 'prompt_time': 0.002175191, 'queue_time': 0.255822479, 'total_time': 0.145811555}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--32ed86ab-3c24-4150-aeae-1d5558b99cbf-0', usage_metadata={'input_tokens': 24, 'output_tokens': 79, 'total_tokens': 103})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Sarthak and I am a AI Software Engineer"),
        AIMessage(content="Hello Sarthak! It's nice to meet you. \n\nAs a AI Software Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Sarthak, and you are an AI Software Engineer.  \n\nRemember, I'm designed to help and be informative, so I keep track of our conversation.  😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 101, 'total_tokens': 142, 'completion_time': 0.074545455, 'prompt_time': 0.005118819, 'queue_time': 0.252268391, 'total_time': 0.079664274}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--99ee6275-ea23-451a-8490-7136f070763b-0', usage_metadata={'input_tokens': 101, 'output_tokens': 41, 'total_tokens': 142})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
!pip install langchain_community

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Sarthak and I am a Software Engineer")],
    config=config
)

In [9]:
response.content

"Hello Sarthak, it's nice to meet you! \n\nI'm glad to know you're a Software Engineer. What kind of software engineering do you specialize in?  \n\nPerhaps you could tell me about a project you're working on or something interesting you've learned recently. 😊  \n\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Sarthak.  😊 \n\nI remember that from our introduction.  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 104, 'total_tokens': 137, 'completion_time': 0.06, 'prompt_time': 0.005147558, 'queue_time': 0.246963161, 'total_time': 0.065147558}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--90b3df82-3d44-4678-bbb6-77998e4dbc67-0', usage_metadata={'input_tokens': 104, 'output_tokens': 33, 'total_tokens': 137})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nIs there anything I can help you with today?\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.  \n\nI remember! 😊  How can I help you further?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sarthak")]})

AIMessage(content="Hi Sarthak!\n\nIt's nice to meet you.  I'm happy to answer any questions you have to the best of my ability.  \n\nWhat can I help you with? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 33, 'total_tokens': 80, 'completion_time': 0.085454545, 'prompt_time': 0.002851498, 'queue_time': 0.260734692, 'total_time': 0.088306043}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--13c2b8a7-2244-4211-82e4-4f902c721362-0', usage_metadata={'input_tokens': 33, 'output_tokens': 47, 'total_tokens': 80})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sarthak")],
    config=config
)

response

AIMessage(content="Hi Sarthak! It's nice to meet you.  \n\nI'm happy to answer any questions you have to the best of my ability.  What can I help you with today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 33, 'total_tokens': 79, 'completion_time': 0.083636364, 'prompt_time': 0.00239287, 'queue_time': 0.24528850300000002, 'total_time': 0.086029234}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f7679f48-7da8-4645-8300-601eb128cd88-0', usage_metadata={'input_tokens': 33, 'output_tokens': 46, 'total_tokens': 79})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sarthak.  I remember! 😊  \n\nIs there anything else I can help you with?\n'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Sarthak")],"language":"Hindi"})
response.content

'नमस्ते Sarthak! 😊 \n\nमुझे आपसे बात करने में खुशी हो रही है। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सही उत्तर दे सकूँ।  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Sarthak")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते सरथक!  😊 \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सही जवाब दूँ। \n\nआप कैसे हैं? \n\n'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [24]:
response.content

'आपका नाम सरथक है। 😊 \n'

In [25]:
!pip install transformers


### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [26]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\ssart\anaconda3\envs\myenv7\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [27]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences, so I don't know what your favorite ice cream is!\n\nWhat's your favorite flavor? 😊  🍦\n"

In [28]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2"  \n\n\n\n\nLet me know if you want to try another one!\n'

In [29]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [30]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations and don't know your name.  \n\nWould you like to tell me your name? 😊  \n"

In [31]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations.  \n\nCan you tell me the math problem you'd like help with? 😊  I'm ready! \n\n"